## Intro

In this PA, you will practice altering data frame objects in order to solve a riddle.

### Advice for troubleshooting your code

#### Check results often

If a chunk of code runs smoothly without giving you any error or warnings this does not necessarily mean it accomplished the desired task.

It is a good habit to check the results of your code every time you finish a task.

If you created or updated a data frame, make sure your edits did what you hoped.  Use the *Environment* or the `head()` function to investigate your changes.

#### Document your work

In the text before the code chunk, make sure to briefly state what the point of the chunk is.  This will remind later readers - which might be your future self! - what the desired output is.

Use comments in the code to give quick explanations of the goals of complex steps.

#### Two heads are better than one

It can be hard to spot bugs in code that you yourself wrote.t
(Have you ever sent a text message and not spotted an obvious  typo?  Same idea!)

Work with people around you if you can - if something goes wrong, ask a friend to take a peek at your code and see if any glaring errors (like *syntax error*) pop out.

#### Explain your code out loud

The best way to troubleshoot a sneaky bug is to explain out loud each step of your code, and what you hoped to accomplish.

If you are alone, try [Rubber Duck Debugging](https://en.wikipedia.org/wiki/Rubber_duck_debugging)!

#### Google is your friend

The whole of the internet is at your disposal!  Use it early, use it often.

Copy-paste the exact error message into Google. Chances are, somebody else had a similar problem and got a similar message.


Include package names in your search terms.  For example, "bar plot in plotnine with pandas data frame" is a better search than "bar plot in python".


## Part One: Data import and cleaning

This section will walk you through some steps to clean up today's dataset, so that you can use it more easily in Part Two.  

First, we declare our package dependencies and load the data.

(Note that the data loading function `read_csv` will give you an outpouring of helpful information about the dataset.  If you do not see the word "error", there is nothing to be concerned about.)


In [80]:
import numpy as np
import pandas as pd

# import data set
colleges = pd.read_csv("https://www.dropbox.com/s/bt5hvctdevhbq6j/colleges.csv?dl=1")
colleges.head()

,Unnamed: 0,INSTNM,CITY,STABBR,ZIP,CONTROL,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,REGION,DEP_INC_PCT_H2,MD_INC_WDRAW_2YR_TRANS_YR4_RT,IND_COMP_4YR_TRANS_YR4_RT,SD_EARN_WNE_P10,FEMALE_WDRAW_4YR_TRANS_YR6_RT,LO_INC_COMP_2YR_TRANS_YR3_RT,NOLOAN_COMP_ORIG_YR4_RT,OPENADMP,PELL_COMP_4YR_TRANS_YR3_RT,DEATH_YR2_RT,NOLOAN_UNKN_ORIG_YR2_RT,NOT1STGEN_WDRAW_ORIG_YR6_RT,HI_INC_YR8_N,CUML_DEBT_P90,C100_L4
0,1,Alabama A & M University,Normal,AL,35762,1,0.9027,929.0,9857.0,18236.0,4824.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,University of Alabama at Birmingham,Birmingham,AL,35294-0110,1,0.9181,1195.0,8328.0,19032.0,12866.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Amridge University,Montgomery,AL,36117-3553,2,NaN,NaN,6900.0,6900.0,322.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,University of Alabama in Huntsville,Huntsville,AL,35899,1,0.8123,1322.0,10280.0,21480.0,6917.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Alabama State University,Montgomery,AL,36104-0271,1,0.9787,935.0,11068.0,19396.0,4189.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The code chunks in this section will cause an error and/or do the desired task incorrectly.  (Even the chunks that run without error are not correct!) You will need to find the mistake, and correct it, to complete the intended action.

None of the *text* is incorrect, only the *code* - your goal is to make the code do exactly what the documentation claims it will.

#### 1. Narrow down the columns to variables of interest

There are too many variables in this dataset.  We don't need all of them. We will limit our dataset down to only:

Name of the institution
City, State, and ZIP code of the institution
The Admissions Rate
The average SAT score
The number of undergraduate students
The in and out of state tuitions
Whether the school is public or private
The "REGION" variable.


In [81]:
# add a set of brackets to specify a list of columns
colleges_clean = colleges[['INSTNM', 'CITY', 'STABBR', 'ZIP', 'ADM_RATE', 'SAT_AVG', 'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'UGDS', "CONTROL", 'REGION']]
colleges_clean.head()

,INSTNM,CITY,STABBR,ZIP,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,CONTROL,REGION
0,Alabama A & M University,Normal,AL,35762,0.9027,929.0,9857.0,18236.0,4824.0,1,5
1,University of Alabama at Birmingham,Birmingham,AL,35294-0110,0.9181,1195.0,8328.0,19032.0,12866.0,1,5
2,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,6900.0,6900.0,322.0,2,5
3,University of Alabama in Huntsville,Huntsville,AL,35899,0.8123,1322.0,10280.0,21480.0,6917.0,1,5
4,Alabama State University,Montgomery,AL,36104-0271,0.9787,935.0,11068.0,19396.0,4189.0,1,5


#### 2. Remove observations that aren't needed

Drop the schools that are "private for-profit" (category 3).  We will not need these for the puzzle.


In [82]:
colleges_clean = colleges_clean[(colleges_clean['CONTROL'] == 1) | (colleges_clean["CONTROL"] == 2)]
colleges_clean.head()

,INSTNM,CITY,STABBR,ZIP,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,CONTROL,REGION
0,Alabama A & M University,Normal,AL,35762,0.9027,929.0,9857.0,18236.0,4824.0,1,5
1,University of Alabama at Birmingham,Birmingham,AL,35294-0110,0.9181,1195.0,8328.0,19032.0,12866.0,1,5
2,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,6900.0,6900.0,322.0,2,5
3,University of Alabama in Huntsville,Huntsville,AL,35899,0.8123,1322.0,10280.0,21480.0,6917.0,1,5
4,Alabama State University,Montgomery,AL,36104-0271,0.9787,935.0,11068.0,19396.0,4189.0,1,5


### 3. Adjust variable types: Numeric

Some of the columns of the dataset were automatically understood by the computer to be categorical, due to missing data appearing like strings.

Adjust the appropriate columns to be numeric.


In [83]:
colleges_clean['TUITIONFEE_IN'] = colleges_clean['TUITIONFEE_IN'].astype('float')

colleges_clean['TUITIONFEE_OUT'] = colleges_clean['TUITIONFEE_OUT'].astype('float')

colleges_clean['SAT_AVG'] = colleges_clean['SAT_AVG'].astype('float')

colleges_clean['ADM_RATE'] = colleges_clean['ADM_RATE'].astype('float')
colleges_clean.head()

,INSTNM,CITY,STABBR,ZIP,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,CONTROL,REGION
0,Alabama A & M University,Normal,AL,35762,0.9027,929.0,9857.0,18236.0,4824.0,1,5
1,University of Alabama at Birmingham,Birmingham,AL,35294-0110,0.9181,1195.0,8328.0,19032.0,12866.0,1,5
2,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,6900.0,6900.0,322.0,2,5
3,University of Alabama in Huntsville,Huntsville,AL,35899,0.8123,1322.0,10280.0,21480.0,6917.0,1,5
4,Alabama State University,Montgomery,AL,36104-0271,0.9787,935.0,11068.0,19396.0,4189.0,1,5


### 4. Adjust variable types: Categorical

Similarly, columns of the dataset that are not numbers are automatically understood by the computer to be strings.

We know, however, that some variables are truly *categorical*, in that there are only a few categories (or "levels") that can be values of the column.

Adjust the appropriate columns to be understood as categorical.


In [84]:
colleges_clean['CONTROL'].astype('str')
colleges_clean['REGION'].astype('str')

0       5
1       5
2       5
3       5
4       5
       ..
7050    6
7051    6
7052    6
7053    6
7056    1
Name: REGION, Length: 4089, dtype: object

#### 5. Calculate new variables

Create a new variable called TUITION_DIFF which contains the difference between in and out of state costs.


In [85]:
colleges_clean['TUITION_DIFF'] = (colleges_clean["TUITIONFEE_OUT"] - colleges_clean["TUITIONFEE_IN"])
colleges_clean.head()

,INSTNM,CITY,STABBR,ZIP,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,CONTROL,REGION,TUITION_DIFF
0,Alabama A & M University,Normal,AL,35762,0.9027,929.0,9857.0,18236.0,4824.0,1,5,8379.0
1,University of Alabama at Birmingham,Birmingham,AL,35294-0110,0.9181,1195.0,8328.0,19032.0,12866.0,1,5,10704.0
2,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,6900.0,6900.0,322.0,2,5,0.0
3,University of Alabama in Huntsville,Huntsville,AL,35899,0.8123,1322.0,10280.0,21480.0,6917.0,1,5,11200.0
4,Alabama State University,Montgomery,AL,36104-0271,0.9787,935.0,11068.0,19396.0,4189.0,1,5,8328.0


#### 6. Deal with missing data

Drop all the rows with missing data.

(It is often a terrible idea to simply drop all rows with *any* missing data!  Usually, even if *some* of the information is missing, we don't want to throw out the entire row.  In this example, however, we'll be lazy.)


In [86]:
colleges_clean = colleges_clean.dropna()
colleges_clean.dropna()

,INSTNM,CITY,STABBR,ZIP,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,CONTROL,REGION,TUITION_DIFF
0,Alabama A & M University,Normal,AL,35762,0.9027,929.0,9857.0,18236.0,4824.0,1,5,8379.0
1,University of Alabama at Birmingham,Birmingham,AL,35294-0110,0.9181,1195.0,8328.0,19032.0,12866.0,1,5,10704.0
3,University of Alabama in Huntsville,Huntsville,AL,35899,0.8123,1322.0,10280.0,21480.0,6917.0,1,5,11200.0
4,Alabama State University,Montgomery,AL,36104-0271,0.9787,935.0,11068.0,19396.0,4189.0,1,5,8328.0
5,The University of Alabama,Tuscaloosa,AL,35487-0166,0.5330,1278.0,10780.0,28100.0,32387.0,1,5,17320.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6391,Milligan College,Milligan College,TN,37682,0.5776,1184.0,32500.0,32500.0,806.0,2,5,0.0
6401,Emory University-Oxford College,Oxford,GA,30054,0.2894,1420.0,48502.0,48502.0,979.0,2,5,0.0
6424,Husson University,Bangor,ME,04401-2929,0.8627,1048.0,17561.0,17561.0,2652.0,2,1,0.0
6532,Piedmont International University,Winston-Salem,NC,27101,0.3633,919.0,9480.0,9480.0,336.0,2,5,0.0


## Part Two:  Identify the mystery college

Wow!  Your best friend Ephelia has been accepted to the college of her dreams!  Unfortunately, Ephelia is a very mysterious person, and she won't tell you directly which college this is.  You'll have to use her clues to figure out which school is her dream school.

Clues:

1. This college is located in Region 1.

2. This college's admission rate is in the first quartile for the region.

3. This college charges the same for in- and out-of-state tuition.

4. The average SAT score of this college is an odd number.

5. This college is NOT in New Hampshire or in the city of Boston.

6. More than 3,000 people apply to this college every year.  (Hint: Make a new variable to compute how many people apply.  Assume the size of an incoming class is 1/4 of the total undergraduate population, and remember that not everyone who *applies* is *admitted*.)

7. Ephelia is not going to an Ivy League school.

8. Of the two options remaining at this step, Ephelia will attend the cheaper one.


In [87]:
# college located in region 1
ephelia = colleges_clean[colleges_clean["REGION"] == 1]
ephelia.head()

,INSTNM,CITY,STABBR,ZIP,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,CONTROL,REGION,TUITION_DIFF
578,Albertus Magnus College,New Haven,CT,06511-1189,0.6388,930.0,31140.0,31140.0,1122.0,2,1,0.0
584,University of Bridgeport,Bridgeport,CT,06604-5620,0.5389,969.0,32250.0,32250.0,3129.0,2,1,0.0
585,Central Connecticut State University,New Britain,CT,6050,0.6676,1071.0,10225.0,22914.0,9317.0,1,1,12689.0
590,University of Connecticut,Storrs,CT,6269,0.4784,1313.0,14880.0,36948.0,18930.0,1,1,22068.0
600,University of Hartford,West Hartford,CT,06117-1599,0.8089,1140.0,38910.0,38910.0,4835.0,2,1,0.0


In [88]:
# college admissions rate is in first quartile for the region
first_quantile = ephelia["ADM_RATE"].quantile(0.25)
ephelia = ephelia[ephelia["ADM_RATE"] <= first_quantile]
ephelia.head()


,INSTNM,CITY,STABBR,ZIP,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,CONTROL,REGION,TUITION_DIFF
590,University of Connecticut,Storrs,CT,6269,0.4784,1313.0,14880.0,36948.0,18930.0,1,1,22068.0
625,Trinity College,Hartford,CT,06106-3100,0.3387,1370.0,54770.0,54770.0,2174.0,2,1,0.0
630,Yale University,New Haven,CT,6520,0.0693,1520.0,51400.0,51400.0,5742.0,2,1,0.0
1416,Colby College,Waterville,ME,04901-8840,0.1579,1444.0,53120.0,53120.0,1917.0,2,1,0.0
1500,Amherst College,Amherst,MA,01002-5000,0.1290,1497.0,54310.0,54310.0,1835.0,2,1,0.0


In [89]:
# college charges same for in state out of state
ephelia = ephelia[ephelia["TUITION_DIFF"] == 0.0]
ephelia.head()

,INSTNM,CITY,STABBR,ZIP,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,CONTROL,REGION,TUITION_DIFF
625,Trinity College,Hartford,CT,06106-3100,0.3387,1370.0,54770.0,54770.0,2174.0,2,1,0.0
630,Yale University,New Haven,CT,6520,0.0693,1520.0,51400.0,51400.0,5742.0,2,1,0.0
1416,Colby College,Waterville,ME,04901-8840,0.1579,1444.0,53120.0,53120.0,1917.0,2,1,0.0
1500,Amherst College,Amherst,MA,01002-5000,0.1290,1497.0,54310.0,54310.0,1835.0,2,1,0.0
1505,Babson College,Babson Park,MA,02457-0310,0.2418,1347.0,49664.0,49664.0,2342.0,2,1,0.0


In [93]:
# average SAT score is an odd number
ephelia = ephelia[ephelia["SAT_AVG"] % 2 !=0]
ephelia.head()

,INSTNM,CITY,STABBR,ZIP,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,CONTROL,REGION,TUITION_DIFF
1500,Amherst College,Amherst,MA,01002-5000,0.1290,1497.0,54310.0,54310.0,1835.0,2,1,0.0
1505,Babson College,Babson Park,MA,02457-0310,0.2418,1347.0,49664.0,49664.0,2342.0,2,1,0.0
1512,Bentley University,Waltham,MA,02452-4705,0.4415,1319.0,48000.0,48000.0,4203.0,2,1,0.0
1520,Brandeis University,Waltham,MA,02454-9110,0.3422,1403.0,53537.0,53537.0,3624.0,2,1,0.0
1549,Harvard University,Cambridge,MA,2138,0.0516,1523.0,48949.0,48949.0,7532.0,2,1,0.0


In [94]:
# not in new hampshire or city of boston
ephelia = ephelia[(ephelia["STABBR"] != "NH") | (ephelia["CITY"] != "Boston")]
ephelia.head()

,INSTNM,CITY,STABBR,ZIP,ADM_RATE,SAT_AVG,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS,CONTROL,REGION,TUITION_DIFF
1500,Amherst College,Amherst,MA,01002-5000,0.1290,1497.0,54310.0,54310.0,1835.0,2,1,0.0
1505,Babson College,Babson Park,MA,02457-0310,0.2418,1347.0,49664.0,49664.0,2342.0,2,1,0.0
1512,Bentley University,Waltham,MA,02452-4705,0.4415,1319.0,48000.0,48000.0,4203.0,2,1,0.0
1520,Brandeis University,Waltham,MA,02454-9110,0.3422,1403.0,53537.0,53537.0,3624.0,2,1,0.0
1549,Harvard University,Cambridge,MA,2138,0.0516,1523.0,48949.0,48949.0,7532.0,2,1,0.0


In [ ]:
# More than 3,000 people apply
